# Ensemble dimensions: splitting changepoint detection algorithms along the dimensions

<!-- {{ add_binder_block(page) }} -->

## Introduction

In `ruptures`, change point detection procedures make use of the same cost function along all the dimensions.
The choice of the cost function is critical as it is related to the type of change to find. 
For instance, [CostL2](../user-guide/costs/costl2.md) can detect shifts in the mean, [CostNormal](../user-guide/costs/costnormal.md) can detect shifts in the mean and the covariance structure, [CostAR](../user-guide/costs/costautoregressive.md) can detect shifts in the auto-regressive structure, etc.

However, in many settings, all the dimensions don't have the same type of changes and a single cost function is not able to spot all changes simultaneously.
To cope with this issue, a procedure to study the cost on each dimension independantly is presented here. It is inspired by the paper [[Katser2021]](#Katser2021), where a procedure to merge several cost functions has been introduced.
In a nutshell, different costs along each dimension can be combined to yield an aggregated cost function which is sensitive to several types of changes.
The aggregated cost can then be used with any search method (such as the [window search method](../user-guide/detection/window.md)) to create change point detection algorithm.

This example illustrates the aggregation procedure, also referred to as an ensemble model.
Here, only [CostL2](../user-guide/costs/costl2.md) is considered for all dimensions, but all other costs could be used. The focus is then on the way the costs are combined (see [intersection](#intersection) and [union](#union)).
In addition, the number of changes is assumed to be known by the user.

## Setup

First, we make the necessary imports and generate a multivariate toy signal which contains different mean shifts along the dimensions. Notice that only one changepoint is shared between the two dimensions. 

In [ ]:
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
import ruptures as rpt


# Scaling function
def minmax(array):
    return (array - np.min(array, axis=0)) / (
        np.max(array, axis=0) - np.min(array, axis=0) + 1e-8
    )


# Aggregation functions
def min_(array):
    return np.min(array, axis=1).T


def max_(array):
    return np.max(array, axis=1).T

In [ ]:
bkps = [329, 656, 972, 1291, 1642, 2000]
n_samples = bkps[-1]
bkps_1 = [bkps[0], bkps[1], bkps[4], n_samples]
bkps_2 = [bkps[1], bkps[2], bkps[3], n_samples]

In [ ]:
signal = np.zeros((n_samples, 2))
val_cycle = cycle([0, 1])
for (start, end) in rpt.utils.pairwise([0] + bkps_1):
    signal[start:end, 0] = next(val_cycle)
for (start, end) in rpt.utils.pairwise([0] + bkps_2):
    signal[start:end, 1] = next(val_cycle)

fig, axes = rpt.display(signal, bkps)
axes[0].set_title("Noise free signal")

signal += np.random.normal(size=signal.shape)
fig, axes = rpt.display(signal, bkps)
_ = axes[0].set_title("Toy signal")

The following cell defines a custom cost that computes the [CostL2](../user-guide/costs/costl2.md) on the given dimension. To define a custom cost that would not compute the same cost on all dimensions, an `if` loop relying on the value of `self.dim` can be imagined. 

In [ ]:
from ruptures.base import BaseCost


class MyCost(BaseCost):

    """Custom cost for exponential signals."""

    # The 2 following attributes must be specified for compatibility.
    model = ""
    min_size = 2

    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def fit(self, signal):
        """Set the internal parameter."""
        self.signal = signal[:, self.dim].reshape(-1, 1)
        return self

    def error(self, start, end) -> float:
        """Return the approximation cost on the segment [start:end].

        Args:
            start (int): start of the segment
            end (int): end of the segment

        Returns:
            segment cost

        Raises:
            NotEnoughPoints: when the segment is too short (less than `min_size` samples).
        """
        if end - start < self.min_size:
            raise rpt.exceptions.NotEnoughPoints

        return self.signal[start:end].var(axis=0).sum() * (end - start)

## Looking at the costs along each dimension

Here, the [window search method](../user-guide/detection/window.md) is used. Thus, the scores will be considered instead of the costs.

The following cell shows the scores along each dimension:

In [ ]:
window_size = 200

list_of_costs = [MyCost(dim=0), MyCost(dim=1)]

scores = []

for cost in list_of_costs:
    algo = rpt.Window(width=window_size, custom_cost=cost, jump=1).fit(signal)
    scores.append(algo.score)
scores = np.array(scores).T

# For display purpose
appended_scores = np.append(
    np.ones((window_size // 2, 2)) * float("inf"), scores, axis=0
)
fig, axes = rpt.display(appended_scores, bkps)
_ = axes[0].set_title("Scores along each dimension")

## Intersection

A first way of aggregating the scores is to consider each score along a dimension as being an expert. The idea is then to take the __intersection__ of the experts so that the predicted changepoints are the changepoints where all the experts are confident. This method is useful when the user is interested in collecting the changepoints that have been detected on all dimensions at the same timestep.

In the following cell, the intersection procedure correctly predicts the only common changepoint between the two dimensions. The aggregated score shows clearly the only changepoint to predict.

In [ ]:
bkps_intersection = [bkps[1], n_samples]
n_bkps_intersection = 1

# intersection aggregation
algo.score = min_(minmax(scores))
bkps_intersection_predicted = algo.predict(n_bkps=n_bkps_intersection)

# For display purpose
appended_intersection_scores = np.append(
    np.ones(window_size // 2) * float("inf"), algo.score
)

fig, (ax,) = rpt.display(
    appended_intersection_scores, bkps_intersection, bkps_intersection_predicted
)
_ = ax.set_title("Aggregated score for intersection purpose")

## Union

Another way of aggregating the scores is to take the __union__ of the experts so that the predicted changepoints are the changepoints where at least one expert is confident. This method is useful when the user is interested in collecting all the changepoints that are present along all dimensions.

In the following cell, the union procedure correctly predicts all the changepoints. The aggregated score shows 5 clear peaks from which to take the changepoints.

In [ ]:
bkps_union = bkps
n_bkps_union = 5

# union aggregation
algo.score = max_(minmax(scores))
bkps_union_predicted = algo.predict(n_bkps=n_bkps_union)

# For display purpose
appended_union_scores = np.append(np.ones(window_size // 2) * float("inf"), algo.score)

fig, (ax,) = rpt.display(appended_union_scores, bkps_union, bkps_union_predicted)
_ = ax.set_title("Aggregated score for union purpose")

## Conclusion

This example shows a way of crafting a changepoint detection algorithm at the scale of the dimensions. This is a finer scale than the usual one: the signal scale.

Two options are presented: [intersection](#intersection) where a changepoint is detected only if it is detected along all dimensions and [union](#union) where a changepoint is detected as soon as one dimension has detected it.

## Authors

This example notebook has been authored by [Théo VINCENT](https://github.com/theovincent) and edited by [Olivier Boulant](https://github.com/oboulant) and [Charles Truong](https://github.com/deepcharles).

## References

<a id="Katser2021">[Katser2021]</a>
Katser, I., Kozitsin, V., Lobachev, V., & Maksimov, I. (2021). Unsupervised Offline Changepoint Detection Ensembles. Applied Sciences, 11(9), 4280.